In [2]:
# get all the libraries needed
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup as b

In [3]:
# make a request to https://www.worldometers.info/coronavirus/#countries
# and check if status code is success
r = requests.get('https://www.worldometers.info/coronavirus/#countries')
r.status_code

200

In [4]:
# parse the text
soup = b(r.text, 'lxml')
# print(soup.prettify())

In [5]:
# find our table
table = soup.find('table', id = 'main_table_countries_today')
print(table)

<table class="table table-bordered table-hover main_table_countries" id="main_table_countries_today" style="width:100%;margin-top: 0px !important;">
<thead>
<tr>
<th width="100">Country,<br/>Other</th>
<th width="20">Total<br/>Cases</th>
<th width="30">New<br/>Cases</th>
<th width="30">Total<br/>Deaths</th>
<th width="30">New<br/>Deaths</th>
<th width="30">Total<br/>Recovered</th>
<th width="30">Active<br/>Cases</th>
<th width="30">Serious,<br/>Critical</th>
<th width="30">Tot Cases/<br/>1M pop</th>
<th width="30">Deaths/<br/>1M pop</th>
<th width="30">Total<br/>Tests</th>
<th width="30">Tests/<br/>
<nobr>1M pop</nobr>
</th>
<th style="display:none" width="30">Continent</th>
</tr>
</thead>
<tbody>
<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>1,187,039</td>
<td>+1,965</td>
<td>69,676</td>
<td>+157</td>
<td>192,470</td>
<td>924,893</td>
<td>16,473</td>
<td></td>
<td></td>

In [6]:
tbody = table.find('tbody')
print(tbody)

<tbody>
<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>1,187,039</td>
<td>+1,965</td>
<td>69,676</td>
<td>+157</td>
<td>192,470</td>
<td>924,893</td>
<td>16,473</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="North America" style="display:none;">North America</td>
</tr>
<tr class="total_row_world row_continent" data-continent="Europe" style="display: none">
<td style="text-align:left;">
<nobr>Europe</nobr>
</td>
<td>1,385,829</td>
<td>+12,385</td>
<td>135,715</td>
<td>+480</td>
<td>511,218</td>
<td>738,896</td>
<td>18,739</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="Europe" style="display:none;">Europe</td>
</tr>
<tr class="total_row_world row_continent" data-continent="Asia" style="display: none">
<td style="text-align:left;">
<nobr>Asia</nobr>
</td>
<td>526,764</td>
<td>+6,714</td>
<td>18,744</td>
<td>+134</td>
<td>267,890</td>
<td>2

In [7]:
table_row = tbody.find_all('tr')
all_rows = []
for tr in table_row:
    td = tr.find_all('td')
    row = [i.text.replace('\n', ' ').strip() for i in td]
    all_rows.append(row)
df = pd.DataFrame(all_rows, columns=['country', 'totalcases', 'newcases', 'totaldeaths', 'newdeaths', 
                                     'totalrecovered', 'active', 'serirous', '1', '2', '3', '4', '5'])        
# print(all_rows)
df.head(10)

,country,totalcases,newcases,totaldeaths,newdeaths,totalrecovered,active,serirous,1,2,3,4,5
0,North America,"1,187,039","+1,965","69,676",+157,"192,470","924,893","16,473",,,,,North America
1,Europe,"1,385,829","+12,385","135,715",+480,"511,218","738,896","18,739",,,,,Europe
2,Asia,"526,764","+6,714","18,744",+134,"267,890","240,130","5,648",,,,,Asia
3,South America,"178,697","+1,883","8,811",+109,"60,118","109,768","9,802",,,,,South America
4,Oceania,"8,348",+16,112,+1,"7,077","1,159",31,,,,,Australia/Oceania
5,Africa,"40,026",+241,"1,640",+2,"13,212","25,174",126,,,,,Africa
6,,721,,15,,645,61,4,,,,,
7,World,"3,327,424","+23,204","234,713",+883,"1,052,630","2,040,081","50,823",427,30.1,,,All
8,USA,"1,095,304",+281,"63,871",+15,"155,737","875,696","15,226","3,309",193,"6,391,887","19,311",North America
9,Spain,"239,639",,"24,543",,"137,984","77,112","2,676","5,125",525,"1,455,306","31,126",Europe


In [8]:
df.drop(index=[0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
df.drop(columns=['1', '2', '3', '4'], inplace=True)
df.head(15)

,country,totalcases,newcases,totaldeaths,newdeaths,totalrecovered,active,serirous,5
8,USA,"1,095,304",+281,"63,871",+15,"155,737","875,696","15,226",North America
9,Spain,"239,639",,"24,543",,"137,984","77,112","2,676",Europe
10,Italy,"205,463",,"27,967",,"75,945","101,551","1,694",Europe
11,UK,"171,253",,"26,771",,N/A,"144,138","1,559",Europe
12,France,"167,178",,"24,376",,"49,476","93,326","4,019",Europe
13,Germany,"163,162",+153,"6,632",+9,"126,900","29,630","2,415",Europe
14,Turkey,"120,204",,"3,174",,"48,886","68,144","1,514",Asia
15,Russia,"114,431","+7,933","1,169",+96,"13,220","100,042","2,300",Europe
16,Iran,"95,646","+1,006","6,091",+63,"76,318","13,237","2,899",Asia
17,Brazil,"87,187","+1,807","6,006",+105,"35,935","45,246","8,318",South America


In [9]:
copy_df = df.copy()
copy_df.head()

,country,totalcases,newcases,totaldeaths,newdeaths,totalrecovered,active,serirous,5
8,USA,"1,095,304",+281,"63,871",+15,"155,737","875,696","15,226",North America
9,Spain,"239,639",,"24,543",,"137,984","77,112","2,676",Europe
10,Italy,"205,463",,"27,967",,"75,945","101,551","1,694",Europe
11,UK,"171,253",,"26,771",,N/A,"144,138","1,559",Europe
12,France,"167,178",,"24,376",,"49,476","93,326","4,019",Europe


In [10]:
copy_df['totalrecovered'] = copy_df['totalrecovered'].str.replace('N/A', '0')
copy_df['newcases'] = copy_df['newcases'].str.replace('+', '')
# copy_df['new cases'] = copy_df['new cases'].str.replace('', '0')

copy_df['new_deaths'] = copy_df['new_deaths'].str.replace('+', '')
# copy_df['new deaths'] = copy_df['new deaths'].str.replace('', '0')
copy_df.head()

KeyError: 'total recovered'

In [68]:
copy_df['total cases'] = copy_df['total cases'].str.replace(',', '')
copy_df['new cases'] = copy_df['new cases'].str.replace(',', '')
copy_df['total deaths'] = copy_df['total deaths'].str.replace(',', '')
copy_df['total recovered'] = copy_df['total recovered'].str.replace(',', '')
copy_df['active'] = copy_df['active'].str.replace(',', '')
copy_df['serirous'] = copy_df['serirous'].str.replace(',', '')
copy_df.head()

,country,total cases,new cases,total deaths,new deaths,total recovered,active,serirous,5
8,USA,1095304,281,63871,15,155737,875696,15226,North America
9,Spain,239639,,24543,,137984,77112,2676,Europe
10,Italy,205463,,27967,,75945,101551,1694,Europe
11,UK,171253,,26771,,0,144138,1559,Europe
12,France,167178,,24376,,49476,93326,4019,Europe


In [69]:
copy_df['total cases'] = pd.to_numeric(copy_df['total cases'])
copy_df['new cases'] = pd.to_numeric(copy_df['new cases'])
copy_df['total deaths'] = pd.to_numeric(copy_df['total deaths'])
copy_df['new deaths'] = pd.to_numeric(copy_df['new deaths'])
copy_df['total recovered'] = pd.to_numeric(copy_df['total recovered'])
copy_df['active'] = pd.to_numeric(copy_df['active'])
copy_df['serirous'] = pd.to_numeric(copy_df['serirous'])
copy_df.head()

,country,total cases,new cases,total deaths,new deaths,total recovered,active,serirous,5
8,USA,1095304,281.0,63871.0,15.0,155737.0,875696,15226.0,North America
9,Spain,239639,NaN,24543.0,NaN,137984.0,77112,2676.0,Europe
10,Italy,205463,NaN,27967.0,NaN,75945.0,101551,1694.0,Europe
11,UK,171253,NaN,26771.0,NaN,0.0,144138,1559.0,Europe
12,France,167178,NaN,24376.0,NaN,49476.0,93326,4019.0,Europe


In [63]:
# copy_df['new cases'] = pd.to_numeric(copy_df['new cases'])
# copy_df['new deaths'] = pd.to_numeric(copy_df['new deaths'])
copy_df.head()

,country,total cases,new cases,total deaths,new deaths,total recovered,active,serirous,1,2,3,4,5
0,North America,1187036,1962.0,69676.0,157,192456.0,924904,16478.0,,,,,North America
1,Europe,1382678,9234.0,135481.0,246,509599.0,737598,18733.0,,,,,Europe
2,Asia,523889,3839.0,18665.0,55,266405.0,238819,5721.0,,,,,Asia
3,South America,178695,1881.0,8811.0,109,60114.0,109770,9802.0,,,,,South America
4,Oceania,8347,15.0,112.0,1,7070.0,1165,34.0,,,,,Australia/Oceania


In [72]:
copy_df.fillna(0, inplace=True)
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 8 to 221
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          214 non-null    object 
 1   total cases      214 non-null    int64  
 2   new cases        214 non-null    float64
 3   total deaths     214 non-null    float64
 4   new deaths       214 non-null    float64
 5   total recovered  214 non-null    float64
 6   active           214 non-null    int64  
 7   serirous         214 non-null    float64
 8   5                214 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory usage: 16.7+ KB


In [11]:
def table():
    r = requests.get('https://www.worldometers.info/coronavirus/#countries')
    print(r.status_code)
    
    soup = b(r.text, 'lxml')
    
    # find our table
    table = soup.find('table', id = 'main_table_countries_today')
    tbody = table.find('tbody')
#     print(tbody)
    
    table_row = tbody.find_all('tr')
    all_rows = []
    for tr in table_row:
        td = tr.find_all('td')
        row = [i.text.replace('\n', ' ').strip() for i in td]
        all_rows.append(row)
        
    df = pd.DataFrame(all_rows, columns=['country', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 
                                     'total_recovered', 'active', 'serirous', '1', '2', '3', '4', '5']) 
#     print(df.head())
    
    df.drop(index=[0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
    df.drop(columns=['1', '2', '3', '4'], inplace=True)
    
    copy_df = df.copy()
#     print(copy_df.head())
    
    copy_df['total_recovered'] = copy_df['total_recovered'].str.replace('N/A', '0')
    copy_df['new_cases'] = copy_df['new_cases'].str.replace('+', '')
    copy_df['new_deaths'] = copy_df['new_deaths'].str.replace('+', '')
#     print(copy_df.head())
    
    copy_df['total_cases'] = copy_df['total_cases'].str.replace(',', '')
    copy_df['new_cases'] = copy_df['new_cases'].str.replace(',', '')
    copy_df['total_deaths'] = copy_df['total_deaths'].str.replace(',', '')
    copy_df['total_recovered'] = copy_df['total_recovered'].str.replace(',', '')
    copy_df['active'] = copy_df['active'].str.replace(',', '')
    copy_df['serirous'] = copy_df['serirous'].str.replace(',', '')
#     print(copy_df.head())
    
    copy_df['total_cases'] = pd.to_numeric(copy_df['total_cases'])
    copy_df['new_cases'] = pd.to_numeric(copy_df['new_cases'])
    copy_df['total_deaths'] = pd.to_numeric(copy_df['total_deaths'])
    copy_df['new_deaths'] = pd.to_numeric(copy_df['new_deaths'])
    copy_df['total_recovered'] = pd.to_numeric(copy_df['total_recovered'])
    copy_df['active'] = pd.to_numeric(copy_df['active'])
    copy_df['serirous'] = pd.to_numeric(copy_df['serirous'])
#     print(copy_df.head())
    
    copy_df.fillna(0, inplace=True)
#     print(copy_df.head())
    
    return copy_df

In [12]:
data = table()
data.head()

200


,country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,active,serirous,5
8,USA,1095977,954.0,63876.0,20.0,155737.0,876364,15226.0,North America
9,Spain,239639,0.0,24543.0,0.0,137984.0,77112,2676.0,Europe
10,Italy,205463,0.0,27967.0,0.0,75945.0,101551,1694.0,Europe
11,UK,171253,0.0,26771.0,0.0,0.0,144138,1559.0,Europe
12,France,167178,0.0,24376.0,0.0,49476.0,93326,4019.0,Europe


In [115]:
boders = gpd.read_file('../TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp')
countries = boders[['NAME', 'ISO3', 'LON', 'LAT', 'geometry']]
countries.rename(columns={'NAME': 'country'}, inplace=True)
countries.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   country   246 non-null    object  
 1   ISO3      246 non-null    object  
 2   LON       246 non-null    float64 
 3   LAT       246 non-null    float64 
 4   geometry  246 non-null    geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 9.7+ KB


/home/gitua/.virtualenvs/pydata/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [116]:
df_merge_col = pd.merge(data, countries, on='country')
df_merge_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   country          178 non-null    object  
 1   total cases      178 non-null    int64   
 2   new cases        178 non-null    float64 
 3   total deaths     178 non-null    float64 
 4   new deaths       178 non-null    float64 
 5   total recovered  178 non-null    float64 
 6   active           178 non-null    int64   
 7   serirous         178 non-null    float64 
 8   5                178 non-null    object  
 9   ISO3             178 non-null    object  
 10  LON              178 non-null    float64 
 11  LAT              178 non-null    float64 
 12  geometry         178 non-null    geometry
dtypes: float64(7), geometry(1), int64(2), object(3)
memory usage: 19.5+ KB


In [131]:
lst = [countries['country'].where(countries['country'] == 'USA')]
for index in lst:
    print(index)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
241    NaN
242    NaN
243    NaN
244    NaN
245    NaN
Name: country, Length: 246, dtype: object
